#Loading Libraries

In [1]:
import pydub
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.io.wavfile import read, write

C:\Users\HakiM\AppData\Local\Programs\Python\Python36\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


#Loading the dataset

In [2]:
#training dataset
train_path  = '../dataset/wavLITE/train/'
errors = []
complete = 0
dataset = []
labels = []
for filename in os.listdir(train_path+'true/'):
    try:
        fs, y = read(train_path+'true/'+filename)
        dataset.append(np.array(y[:10000000]))
        labels.append(1)
        complete = complete + 1
        if len(y) < 10000000:
            print(filename)
    except:
        errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(train_path+'true/')),'%')

complete = 0
for filename in os.listdir(train_path+'false/'):
    try:
        fs, y = read(train_path+'false/'+filename)
        dataset.append(np.array(y[:10000000]))
        labels.append(1)
        complete = complete + 1
        if len(y) < 10000000:
            print(filename)
    except:
        errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(train_path+'false/')),'%')

print("Script complete with ", len(errors), " errors.")
if len(errors):
    for error in errors:
        print(error)

Complete:  2.127659574468085 %
Complete:  4.25531914893617 %
Complete:  6.382978723404255 %
Complete:  8.51063829787234 %
Complete:  10.638297872340425 %
Complete:  12.76595744680851 %
Complete:  14.893617021276595 %
Complete:  17.02127659574468 %
Complete:  19.148936170212767 %
Complete:  21.27659574468085 %
Complete:  23.404255319148938 %
Complete:  25.53191489361702 %
Complete:  27.659574468085108 %
Complete:  29.78723404255319 %
Complete:  31.914893617021278 %
Complete:  34.04255319148936 %
Complete:  36.170212765957444 %
Complete:  38.297872340425535 %
Complete:  40.42553191489362 %
Complete:  42.5531914893617 %
Complete:  44.680851063829785 %
Complete:  46.808510638297875 %
Complete:  48.93617021276596 %
Complete:  51.06382978723404 %
Complete:  53.191489361702125 %
Complete:  55.319148936170215 %
Complete:  57.4468085106383 %
Complete:  59.57446808510638 %
Complete:  61.702127659574465 %
Complete:  63.829787234042556 %
Complete:  65.95744680851064 %
Complete:  68.08510638297872 

In [3]:
#test dataset
test_data = []
test_labels = []
test_path  = '../dataset/wavLITE/test/'

complete = 0
for filename in os.listdir(test_path+'true/'):
    try:
        fs, y = read(test_path+'true/'+filename)
        test_data.append(np.array(y[:10000000]))
        test_labels.append(1)
        complete = complete + 1
        if len(y) < 10000000:
            print(filename)
    except:
        errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(test_path+'true/')),'%')

complete = 0
for filename in os.listdir(test_path+'false/'):
    try:
        fs, y = read(test_path+'false/'+filename)
        test_data.append(np.array(y[:10000000]))
        test_labels.append(0)
        complete = complete + 1
        if len(y) < 10000000:
            print(filename)
    except:
        errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(test_path+'false/')),'%')

print("Script complete with ", len(errors), " errors.")
if len(errors):
    for error in errors:
        print(error)

Complete:  20.0 %
Complete:  40.0 %
Complete:  60.0 %
Complete:  80.0 %
Complete:  100.0 %
Complete:  25.0 %
Complete:  50.0 %
Complete:  75.0 %
Complete:  100.0 %
Script complete with  0  errors.


#Preprocess data

In [4]:
#Convert dataset list of numpy arrays to list of tensors
for i in range(len(dataset)):    
    dataset[i] = tf.convert_to_tensor(dataset[i])
    labels[i] = tf.convert_to_tensor(dataset[i])

for i in range(len(test_data)):    
    test_data[i] = tf.convert_to_tensor(test_data[i])
    test_labels[i] = tf.convert_to_tensor(dataset[i])



In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((dataset, labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

In [6]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [11]:
print(train_dataset)

<BatchDataset shapes: ((None, 10000000, 2), (None, 10000000, 2)), types: (tf.int16, tf.int16)>


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(10000000, 2)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

InvalidArgumentError: Shape [100000000000000,10000000] would have more than 2**63 - 1 elements [Op:RandomUniform]

In [8]:
model.fit(train_dataset, epochs=10)

Epoch 1/10


ValueError: in user code:

    C:\Users\HakiM\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\HakiM\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\HakiM\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\HakiM\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\HakiM\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replic